In [13]:
# dataset loader
from sklearn import datasets

# model training and evalutation utilities 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold # this is one way to generate folds
from sklearn.model_selection import KFold

# models
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

# toy data
X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

## What you should learn/be aware of based on this lecture 

Key `sklearn` functions:
- `train_test_split`
- `cross_validate`
- Fold generators: `KFold` and `StratifiedKFold`
- Scoring functions per last lecture and how to pass to `cross_validate`
- How to compare different models by looping over them with `cross_validate`, `GridSearchCV`, or `RandomizedSearchCV` 

Not covered today but you should check out:
- `confusion_matrix` and `classification_report` (helpful to evaluate models)


## A simple "split, train, evaluate" example

In [14]:
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
# ignore the model I choose here, its not important what
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X1, y1) # fit on the "training data" X1 and  y1

# evaluate the model on the second set of data
y2_model = model.predict(X2) # using X2 (out-of-sample data), predict y2
accuracy_score(y2, y2_model) # see how close y2 is to prediction (fraction of all pred that are exactly right)

0.9066666666666666

## Want to do k-fold? It's like repeating the above. In pseudo code, it looks like:
1. Break the X and y data into $k$ subsamples
2. For each subsample, fit the model, predict OOS, score predictions, and save those

Ok?


## K-Fold in Python: The explicit way, and the wrapped way

Watch me do the explicit way

In [27]:
# you can take quick notes here, but I'm not going to write this code slow enough to copy
# the point here is to illustrate

for train_index, test_index in StratifiedKFold(n_splits=5).split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    print(accuracy_score(y_test, y_predict))

0.9666666666666667
0.9666666666666667
0.9333333333333333
0.9333333333333333
1.0


Now try the wrapper below! We are going to see how to use that function to:
- try multiple models
- try different sets of X variables
- try different ways to specific folds

In [31]:
# try the function here
import numpy as np
np.random.seed(0)
cross_validate(model,X,y,cv=StratifiedKFold(n_splits=5),scoring='accuracy')

{'fit_time': array([0.00150013, 0.00058293, 0.00048399, 0.00078201, 0.00090289]),
 'score_time': array([0.00406194, 0.00208783, 0.00230408, 0.00262713, 0.00202227]),
 'test_score': array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])}

In [26]:
# try here with diff scores
cross_validate(model,X,y,cv=5,scoring=['r2','accuracy','f1_macro'])

cross_validate(model,X,y,cv=5,scoring=['r2','accuracy','f1_macro']) ['test_accuracy'].mean()

0.96

All the metrics it can compute out of the box are here: https://scikit-learn.org/stable/modules/model_evaluation.html

Notice that many of these were discussed in our last lecture!

_**Warning/Note:**_ the metric names on that link and what you put in the `scoring` dictionary don't seem to match up.  

## question:

In [ ]:
# answer here


## Exploring the `cross_validate` parameters

### The model parameter 

In [ ]:
# change the model

### question:


In [20]:
# answer here
cross_validate(linear_model.RidgeClassifier(),X,y,cv=5,scoring=['r2','accuracy','f1_macro'])

{'fit_time': array([0.04941082, 0.00284791, 0.00223732, 0.00293207, 0.00345016]),
 'score_time': array([0.00488305, 0.00206017, 0.0018518 , 0.00185585, 0.00147772]),
 'test_r2': array([0.65, 0.75, 0.75, 0.65, 0.85]),
 'test_accuracy': array([0.76666667, 0.83333333, 0.83333333, 0.76666667, 0.9       ]),
 'test_f1_macro': array([0.76129582, 0.82949702, 0.82949702, 0.76608187, 0.89974937])}

`linear_model` submodule contains lots of useful alternate options

In [ ]:
# for example:
linear_model.Lasso
linear_model.Ridge
linear_model.LogisticRegression

linear_model.LassoCV() # Returns a Lasso (L1 Regularization) linear model with picking the best model by cross validation
linear_model.RidgeCV() # Returns a Ridge (L2 Regularization) linear model with picking the best model by cross validation
linear_model.LogisticRegressionCV() # return best logit model by CV


Looping over models

In [ ]:
# set up models to try
models = []
models.append(('svc_1', SVC(gamma='auto') ))
models.append(('svc_2', SVC(C=5) ))
models.append(('neighbor',  KNeighborsClassifier(n_neighbors=1)))

# loop and print
for name, model in models:
    scores = cross_validate(model, X, y, scoring='accuracy')
    print('%s: %.3f (%.3f)' % (name.ljust(10), 
                                   scores['test_score'].mean(), 
                                   scores['test_score'].std()
                                   )
         )


### The X parameter

You can loop over Xs

In [ ]:
# define a smaller X and a bigger X
X_small = X[:,:2] # just first two columns

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
X3 = poly.fit_transform(X)

# set up Xs to try
right here!

# loop and print
right here!

### Xs and Models

### CV parameter and folds

Just  watch.

In [24]:
import sklearn 
sklearn.__version__

'0.22.1'

# Links, resoruces, and next week

Only two resources needed
- sklearn docs are GREAT https://scikit-learn.org/stable/user_guide.html 
- Python Data Science Handbook (note some module calls are obsolete, so you might need to update code) https://jakevdp.github.io/PythonDataScienceHandbook/index.html

Next week:
- preprocessing
- data transformations
- feasture selection
